# Introduction

|Given|Explanation|
|---|---|
|Main Objective| Use the provided King County Housing data to address a clients need.|
|Client| Erin Robinson
|Client assignment| Find properties in poor neighborhoods that I can invest in (buy, sell) in a socially responsible way. I want my costs back plus a little profit.

### Data set and thought process
King County Housing data set with $N = 21597$ properties where $n = 21420$ are individual properties ($n = 176$ properties were sold more than once).
The [data set has already been cleaned](cleaning_data.ipynb) for the specific chosen clients assignment at hand.

Since there was no further definition given for *poor neighborhoods*, I defined them as:<br>
> *poor neighborhoods* = Zipcode areas where at least 80% of the properties' prices fall into the second quartile.

I.e. the maximum buying price would be the median price over all properties. For all analysis and graphic depictions I used this sub dataset of poor neighborhoods according to this definition. Data cleaning process as well as some premature data exploration can be found in the [cleaning_data.ipynb](cleaning_data.ipynb).

### Hypothesis
The following hypotheses were chosen in order to decide on what properties to focus on within the poor neighborhood areas once those were found.
1. The lower the current condition, the lower the price
2. The farther away from the city center, the lower the price
3. The longer ago the last renovation, the lower the price

# Setting working environment

In [32]:
# IMPORT LIBRARIES
import pandas as pd   
import numpy as np
import plotly.express as px
import folium               # for heatmap on streetmap
from folium import plugins
import warnings
warnings.filterwarnings("ignore")

In [33]:
# IMPORT DATA AS DATA FRAME
df = pd.read_csv('data/kch_poor_neighborhood_clean_data.csv')

# drop column with old indices
df.drop(labels=df.iloc[:,[0]], axis=1,inplace=True)
df.head(2)

,id,date,price,condition,yr_built,yr_renovated,zipcode,lat,long,dist_to_seattle,price_sqft_living,multi_sold,date_sold_old,price_sold_old,yrs_since_renovation
0,7129300520,2014-10-13,221900.0,3,1955,0,98178,47.5112,-122.257,11.979331,188.050847,False,NaN,NaN,0
1,1321400060,2014-06-27,257500.0,3,1995,0,98003,47.3097,-122.327,32.968130,150.145773,False,NaN,NaN,0


# EDA

First lets look where the poor neighborhoods lie. Visually inspect whether they are near to each other or if there are distinct areas. How many zipcode areas are still left after data cleaning process?

In [34]:
# Zipcode areas that are left; i.e. are "poor neighborhoods"
df.zipcode.nunique()

22

Make a ${Price/ft^2}$ heatmap to get an overview on spread of properties.
Tools and basic code for heatmap borrowed from [Qingkai](https://qingkaikong.blogspot.com/2016/06/using-folium-3-heatmap.html?m=1). Made adjustments according to my data.

In [35]:
url_base = 'http://server.arcgisonline.com/ArcGIS/rest/services/'
service = 'World_Street_Map/MapServer/tile/{z}/{y}/{x}'
tileset = url_base + service

m = folium.Map(location=[47.60621, -122.33207], zoom_start=10,\
                control_scale = True, tiles=tileset, attr='USGS style')

m.add_child(plugins.HeatMap(zip(df['lat'], df['long'], df['price_sqft_living']), radius = 12))

Interim take away: Seattle city center has no properties that lie in what fits my definition of a poor neighborhood. Most of the properties are not located near the waterfront.

## Distributions and descriptive statistics

How are condition, years since last renovation, total price and ${Price/ft^2}$ of living space distributed over poor neighborhood properties?

First make new data frame to use for calculations with time since last renovation (since there are a lot of properties where I do not know whether they were never renovated or if that information is simply missing).

In [36]:
# make new df (time since last renovation)
df_with_renovation = pd.DataFrame(df.query('yrs_since_renovation!=0'))

In [37]:
# descriptive stats with all properties (with and without information on renovation)
df.loc[:,['price_sqft_living','condition','dist_to_seattle']].describe()

,price_sqft_living,condition,dist_to_seattle
count,6444.000000,6444.000000,6444.000000
mean,173.526051,3.374146,28.780724
std,49.366677,0.607641,8.703509
min,87.588235,1.000000,10.285662
25%,143.044642,3.000000,22.081040
50%,166.666667,3.000000,30.306145
75%,194.044428,4.000000,34.652623
max,791.666667,5.000000,72.044568


In [38]:
# Distribution - Condition
fig = px.histogram(data_frame=df, x='condition', width=700)

fig.update_layout(font=dict(size=16), xaxis_title="Condition", yaxis_title="Count",
                            title='Distribution of "condition" rating of KCH properties')

fig.show()

In [39]:
df['condition'].value_counts()

condition
3    4261
4    1760
5     360
2      57
1       6
Name: count, dtype: int64

[**Condition description**](https://info.kingcounty.gov/assessor/esales/Glossary.aspx?type=r#b)<br>
Most properties in poor neighborhoods have a condition of value "3" (count(3) = 4261, median = 3):
* Average - Some evidence of deferred maintenance and normal obsolescence with age in that a few minor repairs are needed, along with some refinishing. All major components still functional and contributing toward an extended life expectancy. Effective age and utility is standard for like properties of its class and usage.

Some properties in poor neighborhoods have a condition of value "2" (count(2) = 57):
* Fair - Badly worn. Much repair needed. Many items need refinishing or overhauling, deferred maintenance obvious, inadequate building utility and systems all shortening the life expectancy and increasing the effective age.

Properties with conditions 2 and 3 are promising objects to (a) renovate socially responsible, i.e. no unnecessary luxury renovation as would with conditions 4 and 5 upgrading, while (b) not having to invest so much so that the selling price would result in a far too high price to still be socially responsible (as would most probably be with properties of condition 1).

In [40]:
# Distribution - Time since renovation
fig = px.histogram(data_frame=df_with_renovation, x='yrs_since_renovation', width=650)

fig.update_layout(font=dict(size=16), xaxis_title="Time [years]", yaxis_title="Count",
                            title='Distribution of time since renovation of KCH properties')

fig.show()

**Years since last renovation**<br>
There are $n = 128$ properties with information on last renovation. Most of these properties haven't been renovated for around 30 years ($\mu = 29.63, median = 29.50$), so there are enough properties to choose from with information on last renovation. The minority hasn't been renovated for 40+ years.

In [41]:
# Distribution - Distance from Seattle city center
fig = px.histogram(data_frame=df, x='dist_to_seattle', width=600)

fig.update_layout(font=dict(size=16), xaxis_title="Distance [km]", yaxis_title="Count",
                            title='Distribution of distance to Seattle city center<br>of KCH properties')

fig.show()

The majority of the properties is located roughly $35 km$ from Seattle center ($Q3=34.65$). Properties farther away from the center make it most likely that a car is needed, making these a less socially responsible. People with less means should not *need to* have a car (and thus additional costs).

In [42]:
# descriptive statistics for properties with information on renovation
df_with_renovation.loc[:,['yrs_since_renovation','price_sqft_living','condition','dist_to_seattle']].describe()

,yrs_since_renovation,price_sqft_living,condition,dist_to_seattle
count,128.000000,128.000000,128.000000,128.000000
mean,29.632812,198.324066,3.312500,29.967546
std,15.310370,73.028977,0.585252,10.899854
min,8.000000,104.602510,2.000000,11.719657
25%,15.750000,150.332068,3.000000,21.253919
50%,29.500000,185.873246,3.000000,30.196919
75%,39.000000,225.555556,4.000000,36.706339
max,73.000000,634.615385,5.000000,51.824390


Median and mean for condition and distance to Seattle center have not changed notably, while mean and median for $price/ft^2$ are higher (mean +24.80$, median +19.21$) when only properties with information on renovation are considered. Maybe these properties have been renovated not too long ago or are not so old yet to begin with, making them more expensive on average. In the end not only the $price/ft^2$ matters though.

In [43]:
# Distribution of price per sqft living
fig = px.violin(data_frame=df, y='price_sqft_living', width=550, box=True)
fig.update_layout(font=dict(size=16), yaxis_title="price/sqft [$]",
                  title_text='Distribution of Price/sqft (living space)<br>of KCH properties')
fig.show()

**Price per $ft^2$**<br>
Although we are only looking at poor neighborhoods now, there are still some extreme outliers price-wise with the maximum $price/ft^2 = 791.67 \$$ being 4.5 times higher than the average $price/ft^2 = 173.52 \$$. This may be due to my definition of poor neighborhoods, which would have best been defined by income measures. Could also be properties at the border of richer neighborhoods.

In [44]:
# Distribution of price (total) living
fig = px.violin(data_frame=df, y='price', width=550, box=True)
fig.update_layout(font=dict(size=16), yaxis_title="Price [$]",
                  title_text='Distribution of Price of KCH properties')
fig.show()

**Total price**<br>
Consequential to $price/ft^2$, there are some extreme outliers for the total prices. Crucial for the customers decision should be the $price/ft^2$, though.

## Hypothesis Tests
|#|Hypothesis|
|---|---|
|1|The lower the current condition, the lower the price|
|2|The farther away from the city center, the lower the price|
|3|The longer ago the last renovation, the lower the price

### Plot relationships

In [45]:
# bin condition for distinct color scheme
df['condition']                 =  pd.cut(df['condition'], bins=5, labels=[1,2,3,4,5])

df_with_renovation['condition'] =  pd.cut(df_with_renovation['condition'],
                                          bins=df_with_renovation['condition'].nunique(),
                                          labels=df_with_renovation['condition'].unique())

In [46]:
# Create a scatter plot on relationship between price/sqft, time since renovation and condition
fig = px.scatter(data_frame=df_with_renovation, x='yrs_since_renovation', y='price_sqft_living',
                 color='condition', labels=dict(yrs_since_renovation='Years since renovation',
                                               price_sqft_living='Price/sqft [$]'),
                 title='Relationship between price/sqft, time since renovation<br>and condition for KCH '
                 'properties in poor neighborhoods', opacity=.7, width=700,
                 category_orders=dict(condition=[1,2,3,4,5]))

fig.update_layout(font=dict(size=16))

fig.show()

Only from looking at it, there does not seem to be any kind of meaningful correlation between years since renovation and $price/ft^2$. With only few properties with condition 5 and <4, there is not much information to draw from on a possible relationship between condition and $price/ft^2$. From what there is to observe, there doesn't seem to be a clear relationship.

In [47]:
# Create a scatter plot on relationship between price/sqft, distance to Seattle and condition
fig = px.scatter(data_frame=df, x='dist_to_seattle', y='price_sqft_living',
                 color='condition', title='Relationship between price/sqft, distance to Seattle center<br>and condition '
                 'for KCH properties in poor neighborhoods',
                 labels=dict(dist_to_seattle='Distance to Seattle [km]',
                             price_sqft_living='Price/sqft [$]'),
                             opacity=.5, width=700,
                             category_orders=dict(condition=[1,2,3,4,5]))
fig.update_layout(font=dict(size=16))

fig.show()

Most of the data points gather in a point cloud with some outliers. A meaningful correlation seems very unlikely.

### Test for correlations

In [48]:
# condition is categorical, use spearman correlation
df[["price_sqft_living", "condition"]].corr(method='spearman')

,price_sqft_living,condition
price_sqft_living,1.00000,0.05358
condition,0.05358,1.00000


In [49]:
df[["price_sqft_living", "dist_to_seattle"]].corr()

,price_sqft_living,dist_to_seattle
price_sqft_living,1.000000,-0.093797
dist_to_seattle,-0.093797,1.000000


Only for properties with info on renovation

In [50]:
df_with_renovation[["price_sqft_living", "yrs_since_renovation"]].corr()

,price_sqft_living,yrs_since_renovation
price_sqft_living,1.000000,-0.034618
yrs_since_renovation,-0.034618,1.000000


In [51]:
# condition is categorical, use spearman correlation
df_with_renovation[["price_sqft_living", "condition"]].corr(method='spearman')

,price_sqft_living,condition
price_sqft_living,1.000000,0.024974
condition,0.024974,1.000000


# Results

__Overview__
|#|Hypothesis|Result|
|---|---|---|
|1|The lower the current condition, the lower the price|$\rho = .05$|
|2|The farther away from the city center, the lower the price|$r = -.09$|
|3|The longer ago the last renovation, the lower the price|$r = -.03$

* Correlation between price/$ft^2$ and condition is near to not existent.
* Correlation between price/$ft^2$ and distance to Seattle city center is at best marginal.
* Correlation between price/$ft^2$ and time since last renovation is near to not existent.
* Correlation directions for all as hypothesized.

The data for poor neighborhoods are very homogenous. There are some outliers price-wise, but over all not much difference ($\mu = 173.53, median = 166.67, s = 49.34$). All properties are outside of Seattle, so there is not much variability in distance to center. Time since renovation is also very homogenous (right skewed, $\mu = 29.63, s = 15.31, median = 29.50$).
Additionally, there are very few properties with condition ratings of 1 and 2, reducing their weights. 
This might explain why we don't see any stronger correlations.


## Recommendation for Client

Since there are no meaningful correlations, it doesn't seem necessary to exclude further zip code areas in respect to distance to city center.
As for expected (renovation) costs it seems to make sense to look closer into properties with a KCH condition rating of 2 or 3 (1 would most probably become very expensive which would be represented in the later selling price). A further consideration of time since last renovation is only in so far efficient if we focus on properties where we have that information or which are not older than 40 years. Otherwise this information would have to be retrieved from KCH. It should be considered that the longer ago the last renovation, the more probably unforeseen problems (and costs) will arise, independent from KCH condition rating.

I specifically recommend my client, Erin Robinson, to take properties into consideration for an investment that meet the following criteria:
* properties in 22 *poor* zip code areas
* KCH condition of 2 or 3
* last renovation not longer ago than 40 years
* since it should still be affordable and social responsible after renovation:
    $price(total) <= median(price) of all properties$
* Additionally looking for properties with at least 3 bedrooms to meet needs of families (social responsibility).

The following properties meet all of these criteria:

In [52]:
# Need data frame with all information to look for bedroom number. import
kch = pd.read_csv('data/kch_clean_data.csv')

In [53]:
possible_properties = df.query('(condition==2 or condition==3) and (yrs_since_renovation>0 and yrs_since_renovation<=40)')['id'].values

### Final properties - investment possibilities

In [54]:
final_properties = kch.query('id in @possible_properties and price<=price.median() and price_sqft_living<=price_sqft_living.median() and bedrooms>=3'
                             )[['id','date','price','price_sqft_living','sqft_living','bedrooms','bathrooms','yr_renovated','yr_built','floors','sqft_lot','lat','long']]

final_properties.sort_values(by='price_sqft_living', axis=0, ascending=True)

,id,date,price,price_sqft_living,sqft_living,bedrooms,bathrooms,yr_renovated,yr_built,floors,sqft_lot,lat,long
17683,4058800215,2014-05-28,430000.0,110.539846,3890,3,3.75,2007,1943,1.0,7140,47.5073,-122.239
15001,3832700250,2014-09-29,270000.0,110.655738,2440,4,2.75,1985,1963,1.0,7150,47.3662,-122.282
13834,9456200450,2015-04-27,212000.0,111.578947,1900,4,2.50,1987,1940,1.5,21780,47.3776,-122.314
10947,1180002470,2014-11-04,354000.0,117.218543,3020,6,3.50,1992,1941,2.0,4500,47.4980,-122.225
10208,2472950350,2015-04-11,312500.0,125.000000,2500,4,2.50,2008,1984,1.0,11983,47.4292,-122.148
4736,8097000190,2014-06-02,350000.0,130.597015,2680,3,2.50,2009,1990,2.0,7836,47.3203,-122.185
5206,4131500190,2015-05-07,379000.0,135.212273,2803,5,2.50,2011,1963,1.0,8550,47.3032,-122.306
16016,5643600351,2014-08-06,257000.0,135.263158,1900,4,1.75,1990,1922,1.5,22896,47.3102,-122.023
8094,3141600210,2014-06-19,186000.0,138.805970,1340,3,2.00,1993,1912,1.0,4320,47.2990,-122.228
11025,6002400030,2014-08-14,324950.0,140.064655,2320,4,1.75,2014,1959,1.0,9240,47.4909,-122.257


In [55]:
final_properties.loc[:,['price','price_sqft_living','sqft_living','bedrooms','bathrooms','floors','sqft_lot']].describe()

,price,price_sqft_living,sqft_living,bedrooms,bathrooms,floors,sqft_lot
count,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000
mean,287330.416667,170.413996,1766.083333,3.437500,1.911458,1.250000,22151.375000
std,72223.044768,34.053630,626.195973,0.649263,0.732036,0.399468,50337.246988
min,146000.000000,110.539846,840.000000,3.000000,1.000000,1.000000,3000.000000
25%,234000.000000,141.806220,1307.500000,3.000000,1.375000,1.000000,7080.000000
50%,270000.000000,175.000000,1620.000000,3.000000,1.750000,1.000000,9030.000000
75%,332500.000000,191.017617,2142.500000,4.000000,2.500000,1.500000,14995.000000
max,446000.000000,244.186047,3890.000000,6.000000,3.750000,2.500000,273556.000000


In [56]:
url_base = 'http://server.arcgisonline.com/ArcGIS/rest/services/'
service = 'World_Street_Map/MapServer/tile/{z}/{y}/{x}'
tileset = url_base + service

m = folium.Map(location=[47.60621, -122.33207], zoom_start=10,\
                control_scale = True, tiles=tileset, attr='USGS style')

m.add_child(plugins.HeatMap(zip(final_properties['lat'], final_properties['long'], final_properties['price_sqft_living']), radius = 12))